In [6]:
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [7]:
df = pd.read_csv("C:/Users/ASUS/Downloads/HAM10000_metadata.csv")

IMG_SIZE =128

images = []
labels = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    img_file = os.path.join("C:/Users/ASUS/Downloads/HAM10000_images_part_1", row['image_id'] + ".jpg")
    img = cv2.imread(img_file)

    if img is not None:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        images.append(img)

        # Binary label
        if row['dx'] in ['mel', 'bcc', 'akiec']:
            labels.append(1)   # Cancer
        else:
            labels.append(0)   # Non-Cancer

X = np.array(images)
y = np.array(labels)

print("Data Loaded:", X.shape)

100%|██████████| 10015/10015 [01:00<00:00, 166.40it/s]


Data Loaded: (5000, 128, 128, 3)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print("Train:", X_train.shape)

Train: (4000, 128, 128, 3)


In [10]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')   # Binary
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

C:\Users\ASUS\PycharmProjects\data_analysis\ML_clg\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,705,921 (14.14 MB)

 Trainable params: 3,705,921 (14.14 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history=model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
            epochs=5,
          batch_size=32)

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 205ms/step - accuracy: 0.9028 - loss: 0.2200 - val_accuracy: 0.8270 - val_loss: 0.4149
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 177ms/step - accuracy: 0.9195 - loss: 0.1913 - val_accuracy: 0.8370 - val_loss: 0.3861
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 181ms/step - accuracy: 0.9265 - loss: 0.1725 - val_accuracy: 0.8340 - val_loss: 0.4563
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step - accuracy: 0.9283 - loss: 0.1670 - val_accuracy: 0.8320 - val_loss: 0.4744
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 190ms/step - accuracy: 0.9413 - loss: 0.1452 - val_accuracy: 0.8300 - val_loss: 0.4260


In [16]:
model.save("skin_cancer_model.h5")

In [17]:
print(history.history['accuracy'])
print(history.history['val_accuracy'])

[0.9027500152587891, 0.9194999933242798, 0.9265000224113464, 0.9282500147819519, 0.9412500262260437]
[0.8270000219345093, 0.8370000123977661, 0.8339999914169312, 0.8320000171661377, 0.8299999833106995]
